# Data Wrangling and Model Building

## Conclusions From EDA
* Children under 4 have higher transportation rates
* VIP and HomePlanet have interaction: VIP*Europa are transported, VIP*Earth are not transported
* Cryosleep means higher transportaion risk
* HomePlanet affects transportation rates
* Destination affects transportation rates
* VIP means lower transporation risk
* Larger spending on luxury services means lowered transported chance
* Deck affects transported rates

## Wrangling Tasks
* Combine spending into one column (Done)
* Extract deck numbers from Cabin column (Done)
* Drop unwanted uncolumns (Done)
* Extract deck numbers from Cabin column

**Impute missing values by automated procedures!**
* Complete age column using the distribution of exsiting ages
* Complete spending column using the distribution


* Complete VIP and HomePlanet columns by Spending distribution if spending is present
* Complete spending column by VIP and HomePlanet column if VIP and HomePlanet are present; otherwise, by the spending distribution function
* Complete cryosleep
* Complete VIP column by spending

**Create New Indicator Columns**
* Convert string values to ordinal values
* Indicator whether a children is under 4 or not
* Create interaction columns: VIP*Europa, VIP*Earth
* Indicator column for whether the destination is "55 Cancri e"

In [212]:
# for data wrangling
import pandas as pd
import numpy as np

# Visualize Data
import seaborn as sns
import matplotlib.pyplot as plt

# Model, predict, and solve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer

In [182]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
combine = [train, test]
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


## Add spending and Deck column, drop unwanrted columns

In [183]:
for dataset in combine:
    dataset['Spending'] = dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].sum(axis=1)
    dataset['Deck'] = dataset['Cabin'].str.extract(pat=r"^(\w?)", expand=False)

combine[0] = combine[0].drop(['PassengerId', 'Name', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin'], axis = 1)
combine[1] = combine[1].drop(['PassengerId', 'Name', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Cabin'], axis = 1)
combine[0].describe()

,Age,Spending
count,8514.000000,8693.000000
mean,28.827930,1440.866329
std,14.489021,2803.045694
min,0.000000,0.000000
25%,19.000000,0.000000
50%,27.000000,716.000000
75%,38.000000,1441.000000
max,79.000000,35987.000000


In [184]:
train.shape, test.shape

((8693, 16), (4277, 15))

In [185]:
# combine[0].drop(['PassengerId', 'Name', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], axis = 1)
combine[0].describe(include = 'O'), 'Transported' in combine[0].columns.values

(       HomePlanet CryoSleep  Destination    VIP  Deck
 count        8492      8476         8511   8490  8494
 unique          3         2            3      2     8
 top         Earth     False  TRAPPIST-1e  False     F
 freq         4602      5439         5915   8291  2794,
 True)

In [186]:
combine[1].describe(include = 'O')

,HomePlanet,CryoSleep,Destination,VIP,Deck
count,4190,4184,4185,4184,4177
unique,3,2,3,2,8
top,Earth,False,TRAPPIST-1e,False,F
freq,2263,2640,2956,4110,1445


Both training and testing data have missing values

## Impute Categorical Data by Simple Imputer of skLearn

In [187]:
# Separate categorical and numerial columns
cate_col = list(filter(lambda c: test[c].dtype =='object' ,combine[1].columns))
num_col = list(filter(lambda c: test[c].dtype =='float64' ,combine[1].columns))

In [188]:
for i in [0, 1]:
    dataset = combine[i]
    imputer = SimpleImputer(strategy='most_frequent')
    imp_data = imputer.fit_transform(dataset[cate_col])
    imp_data = pd.DataFrame(imp_data, columns = cate_col)
    combine[i][cate_col] = imp_data[cate_col]
    # print(imp_data), print(imp_data.info())

combine[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HomePlanet   8693 non-null   object 
 1   CryoSleep    8693 non-null   object 
 2   Destination  8693 non-null   object 
 3   Age          8514 non-null   float64
 4   VIP          8693 non-null   object 
 5   Transported  8693 non-null   bool   
 6   Spending     8693 non-null   float64
 7   Deck         8693 non-null   object 
dtypes: bool(1), float64(2), object(5)
memory usage: 484.0+ KB


In [189]:
# Simply impute numerical variates
for i in [0, 1]:
    dataset = combine[i]
    imputer = SimpleImputer(strategy='median')
    imp_data = imputer.fit_transform(dataset[num_col])
    imp_data = pd.DataFrame(imp_data, columns = num_col)
    combine[i][num_col] = imp_data[num_col]
    # print(imp_data), print(imp_data.info())

combine[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HomePlanet   8693 non-null   object 
 1   CryoSleep    8693 non-null   object 
 2   Destination  8693 non-null   object 
 3   Age          8693 non-null   float64
 4   VIP          8693 non-null   object 
 5   Transported  8693 non-null   bool   
 6   Spending     8693 non-null   float64
 7   Deck         8693 non-null   object 
dtypes: bool(1), float64(2), object(5)
memory usage: 484.0+ KB


## Indicator whether a children is transproted or not

In [190]:
for i in [0, 1]:
    combine[i]['Children'] = combine[i]['Age'].map(lambda x: 1 if x <= 4 else 0)
combine[0].Children.info()

<class 'pandas.core.series.Series'>
RangeIndex: 8693 entries, 0 to 8692
Series name: Children
Non-Null Count  Dtype
--------------  -----
8693 non-null   int64
dtypes: int64(1)
memory usage: 68.0 KB


## Convert Homeplanet and Destination to ordinal values

In [191]:
for i in [0, 1]:
    combine[i]['HomePlanetCode'] = combine[i]['HomePlanet'].map({'Europa':0, 'Earth':1, 'Mars':2})
    combine[i]['DestinationCode'] = combine[i]['Destination'].map(lambda x: 1 if x== '55 Cancri e' else 0)
combine[0][['HomePlanetCode', 'DestinationCode']].describe()

,HomePlanetCode,DestinationCode
count,8693.000000,8693.000000
mean,0.957207,0.207063
std,0.667613,0.405224
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,0.000000
max,2.000000,1.000000


In [192]:
combine[0][['HomePlanetCode', 'DestinationCode']]

,HomePlanetCode,DestinationCode
0,0,0
1,1,0
2,0,0
3,0,0
4,1,0
...,...,...
8688,0,1
8689,1,0
8690,1,0
8691,0,1


## Add indicators: VIP\*Europa, VIP\*Earth 

In [193]:
for i in [0, 1]:
    combine[i]['VIP*Europa'] = combine[i].apply(lambda x: 1 if (x.VIP and x.HomePlanet == 'Europa') else 0, axis = 1)
    combine[i]['VIP*Earth'] = combine[i].apply(lambda x: 1 if (x.VIP and x.HomePlanet == 'Earth') else 0, axis = 1)

combine[0][['VIP*Europa', 'VIP*Earth']].describe()

,VIP*Europa,VIP*Earth
count,8693.000000,8693.000000
mean,0.015070,0.000575
std,0.121837,0.023977
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,1.000000


## Create Fare Band using ordinal values

In [194]:
FareInterval = pd.cut(combine[0].Spending, bins = combine[0].Spending.quantile([0, 0.5, 0.9, 1]), include_lowest = True)
def FareBand(Fare):
    if Fare in FareInterval[0]:
        return 0
    elif Fare in FareInterval[1]:
        return 1
    else:
        return 2

for i in [0,1]:
    combine[i]['FareBand'] = combine[i]['Spending'].map(FareBand)

combine[0].head()


,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,Spending,Deck,Children,HomePlanetCode,DestinationCode,VIP*Europa,VIP*Earth,FareBand
0,Europa,False,TRAPPIST-1e,39.0,False,False,0.0,B,0,0,0,0,0,0
1,Earth,False,TRAPPIST-1e,24.0,False,True,736.0,F,0,1,0,0,0,1
2,Europa,False,TRAPPIST-1e,58.0,True,False,10383.0,A,0,0,0,1,0,2
3,Europa,False,TRAPPIST-1e,33.0,False,False,5176.0,A,0,0,0,0,0,2
4,Earth,False,TRAPPIST-1e,16.0,False,True,1091.0,F,0,1,0,0,0,1


# Convert Deck to Ordinal Numbers

In [195]:
DeckCode = sorted(combine[0].Deck.unique())
DeckDict = {letter:indx for indx, letter in enumerate(DeckCode)}
for i in [0,1]:
    combine[i]['DeckCode'] = combine[i]['Deck'].map(DeckDict)
combine[0].describe()


,Age,Spending,Children,HomePlanetCode,DestinationCode,VIP*Europa,VIP*Earth,FareBand,DeckCode
count,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000
mean,28.790291,1440.866329,0.053606,0.957207,0.207063,0.015070,0.000575,0.600023,4.321293
std,14.341404,2803.045694,0.225252,0.667613,0.405224,0.121837,0.023977,0.663481,1.760826
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,27.000000,716.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,5.000000
75%,37.000000,1441.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,6.000000
max,79.000000,35987.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,7.000000


In [196]:
for i in [0,1]:
    combine[i] = combine[i].drop(['Spending', 'Age', 'HomePlanet', 'Destination', 'Deck'], axis = 1)

combine[0].describe()

,Children,HomePlanetCode,DestinationCode,VIP*Europa,VIP*Earth,FareBand,DeckCode
count,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000,8693.000000
mean,0.053606,0.957207,0.207063,0.015070,0.000575,0.600023,4.321293
std,0.225252,0.667613,0.405224,0.121837,0.023977,0.663481,1.760826
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,5.000000
75%,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,6.000000
max,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,7.000000


In [197]:
combine[0].describe(include = 'O')

,CryoSleep,VIP
count,8693,8693
unique,2,2
top,False,False
freq,5656,8494


In [198]:
# Store the cleaned data in files
train = combine[0]
test = combine[1]

train.to_csv("train_cleaned.csv")
test.to_csv('test_cleaned.csv')

In [204]:
combine[0].VIP.loc[0] == False

True

## Modelling and Predict 

In [208]:
train_X = train.drop('Transported', axis = 1)
train_y = train.Transported
val_X = test

# Logistic regression attempt
LR = LogisticRegression()
LR.fit(train_X, train_y)
logit_pred  = LR.predict(val_X)
round(LR.score(train_X, train_y)*100, 2)

73.35

In [215]:
# Decision Tree
DT = DecisionTreeClassifier()
DT.fit(train_X, train_y)
dt_pred  = DT.predict(val_X)
round(DT.score(train_X, train_y)*100, 2)

74.99

In [213]:
# Random Forest
RF = RandomForestClassifier()
RF.fit(train_X, train_y)
rf_pred  = RF.predict(val_X)
round(RF.score(train_X, train_y)*100, 2)

74.99

In [216]:
# KNN Method
KNN = KNeighborsClassifier()
KNN.fit(train_X, train_y)
dt_pred  = KNN.predict(val_X)
round(KNN.score(train_X, train_y)*100, 2)

67.46

In [218]:
Sample_subsmission = pd.read_csv('sample_submission.csv')
Sample_subsmission.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [224]:
# Logistic submission
Sample_subsmission.Transported = logit_pred
Sample_subsmission.to_csv("logit_submission.csv")
Sample_subsmission.Transported.values

array([ True, False,  True, ...,  True, False,  True])

In [223]:
Sample_subsmission.Transported = logit_pred
Sample_subsmission.to_csv("decision_tree_submission.csv")
Sample_subsmission.Transported.values

array([ True, False,  True, ...,  True, False,  True])